In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def view_detections():    
    cap = cv2.VideoCapture(0)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [7]:
#extracting the x,y co-ordinates of landmarks detected by mediapipe and storing in numpy arrays, here left and right hands with the posture
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,left_hand,right_hand])

In [29]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect | null implies none of the meaningful actions were detected in current frames
actions = np.array(['null','apply-throttle', 'release-throttle', 'apply-brake','release-brake'])

#next batch additional size
seq_batch = 5

#the size of last sequence of videos batch
dirmax = np.max(np.array([eval(i) for i in os.listdir(os.path.join(DATA_PATH, action))]))

# Thirty videos worth of data
sequence_limit = dirmax + seq_batch + 1

# Videos are going to be 30 frames in length
frame_limit = 30


In [30]:
#This block creates the respective folders to store the data(the numpy files)
# each action(ex: throttle-apply) -> video(sequence) -> frames(each one is a .np file containing x,y coordinates of the landmarks)
for action in actions:
    for sequence in range(dirmax+1,sequence_limit):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [39]:
actions[2:3]

array(['release-throttle'], dtype='<U16')

In [ ]:
cap = cv2.VideoCapture(0)

is_recording = False  # Flag to track recording state
is_done = False  # Flag to track the initial pause

# Keyboard event handler
def key_press(key):
    global is_recording,is_done
    #click s to set specific flags
    if key == ord(' '):
        is_done = False
        is_recording = True

#copying the actions array to a temp array to slice to train for a few actions if required 
actions_temp = actions.copy()
actions_temp = actions_temp[2:3]
#Start value for sequence
sequence_limit = 20
start_seq = 15
#Intial values of frame and video number
sequence = start_seq
frame_num = 0
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # Display "Collecting..." message after pressing 's'
        if frame_num >=1 and is_recording:
            cv2.putText(image, "Collecting - frame {}".format(frame_num), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4,cv2.LINE_AA)
        if is_done:
            cv2.putText(image, 'action: {} | sequence: {} '.format(actions_temp[-1] if len(actions_temp) else 'empty',sequence), (20,20), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.632, (0,255, 0), 4, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)
        #taking input from user
        key_selected = cv2.waitKey(1)
        # Store landmarks when 's' key is pressed
        key_press(key_selected)
        #click "s" to initiate video/frame collection
        if  is_recording and len(actions)>0:
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, actions_temp[-1], str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    print('action: {} | sequence: {} | frame: {} '.format(actions_temp[-1] if len(actions_temp) else 'empty',sequence,frame_num))
                    frame_num += 1
                    if frame_num == frame_limit:
                        frame_num=0
                        is_recording = False
                        is_done = True
                        sequence+=1
                    if sequence==sequence_limit:
                        sequence=start_seq
                        actions_temp = actions_temp[:-1]
        elif key_selected == ord('q'):
            break
        else:
            pass
        
    cap.release()
    cv2.destroyAllWindows()

In [41]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [42]:
actions_temp = actions.copy()
actions_temp = actions_temp[0:3]
label_map = {label:num for num, label in enumerate(actions_temp)}

In [43]:
label_map

{'null': 0, 'apply-throttle': 1, 'release-throttle': 2}

In [45]:
sequences, labels = [], []
for action in actions_temp:
    for sequence in range(sequence_limit):
        window = []
        for frame_num in range(frame_limit):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [46]:
X = np.array(sequences)

In [47]:
y = to_categorical(labels).astype(int)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [49]:
X.shape

(60, 30, 258)

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [51]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [52]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions_temp.shape[0], activation='softmax'))

In [53]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.fit(X_train, y_train, epochs=540, callbacks=[tb_callback])

Epoch 1/540
2/2 [==============================] - 0s 39ms/step - loss: 1.0979 - categorical_accuracy: 0.3509
Epoch 2/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0979 - categorical_accuracy: 0.3509
Epoch 3/540
2/2 [==============================] - 0s 41ms/step - loss: 1.0984 - categorical_accuracy: 0.3509
Epoch 4/540
2/2 [==============================] - 0s 38ms/step - loss: 1.0978 - categorical_accuracy: 0.3509
Epoch 5/540
2/2 [==============================] - 0s 39ms/step - loss: 1.0981 - categorical_accuracy: 0.3509
Epoch 6/540
2/2 [==============================] - 0s 39ms/step - loss: 1.0979 - categorical_accuracy: 0.3509
Epoch 7/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0977 - categorical_accuracy: 0.3509
Epoch 8/540
2/2 [==============================] - 0s 38ms/step - loss: 1.0976 - categorical_accuracy: 0.3509
Epoch 9/540
2/2 [==============================] - 0s 38ms/step - loss: 1.0977 - categorical_accuracy: 0.3509
Epoch 10/5

2/2 [==============================] - 0s 43ms/step - loss: 1.0974 - categorical_accuracy: 0.3509
Epoch 75/540
2/2 [==============================] - 0s 44ms/step - loss: 1.0973 - categorical_accuracy: 0.3509
Epoch 76/540
2/2 [==============================] - 0s 42ms/step - loss: 1.0973 - categorical_accuracy: 0.3509
Epoch 77/540
2/2 [==============================] - 0s 48ms/step - loss: 1.0973 - categorical_accuracy: 0.3509
Epoch 78/540
2/2 [==============================] - 0s 47ms/step - loss: 1.0975 - categorical_accuracy: 0.3509
Epoch 79/540
2/2 [==============================] - 0s 48ms/step - loss: 1.0973 - categorical_accuracy: 0.3509
Epoch 80/540
2/2 [==============================] - 0s 58ms/step - loss: 1.0972 - categorical_accuracy: 0.3509
Epoch 81/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0973 - categorical_accuracy: 0.3509
Epoch 82/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0972 - categorical_accuracy: 0.3509
Epoch 83/540
2

2/2 [==============================] - 0s 74ms/step - loss: 1.0973 - categorical_accuracy: 0.3684
Epoch 148/540
2/2 [==============================] - 0s 42ms/step - loss: 1.0967 - categorical_accuracy: 0.3684
Epoch 149/540
2/2 [==============================] - 0s 43ms/step - loss: 1.0963 - categorical_accuracy: 0.3684
Epoch 150/540
2/2 [==============================] - 0s 49ms/step - loss: 1.0961 - categorical_accuracy: 0.3684
Epoch 151/540
2/2 [==============================] - 0s 43ms/step - loss: 1.0967 - categorical_accuracy: 0.3684
Epoch 152/540
2/2 [==============================] - 0s 46ms/step - loss: 1.0960 - categorical_accuracy: 0.3684
Epoch 153/540
2/2 [==============================] - 0s 42ms/step - loss: 1.0961 - categorical_accuracy: 0.3684
Epoch 154/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0962 - categorical_accuracy: 0.3684
Epoch 155/540
2/2 [==============================] - 0s 57ms/step - loss: 1.0959 - categorical_accuracy: 0.3684
Epoch 

2/2 [==============================] - 0s 39ms/step - loss: 1.0168 - categorical_accuracy: 0.5263
Epoch 221/540
2/2 [==============================] - 0s 46ms/step - loss: 1.0325 - categorical_accuracy: 0.4737
Epoch 222/540
2/2 [==============================] - 0s 38ms/step - loss: 1.0971 - categorical_accuracy: 0.3509
Epoch 223/540
2/2 [==============================] - 0s 45ms/step - loss: 1.0492 - categorical_accuracy: 0.4211
Epoch 224/540
2/2 [==============================] - 0s 40ms/step - loss: 1.0272 - categorical_accuracy: 0.5088
Epoch 225/540
2/2 [==============================] - 0s 38ms/step - loss: 1.0019 - categorical_accuracy: 0.4737
Epoch 226/540
2/2 [==============================] - 0s 39ms/step - loss: 0.9599 - categorical_accuracy: 0.5439
Epoch 227/540
2/2 [==============================] - 0s 46ms/step - loss: 1.0803 - categorical_accuracy: 0.3684
Epoch 228/540
2/2 [==============================] - 0s 42ms/step - loss: 0.9774 - categorical_accuracy: 0.5088
Epoch 

2/2 [==============================] - 0s 39ms/step - loss: 0.8432 - categorical_accuracy: 0.6316
Epoch 294/540
2/2 [==============================] - 0s 38ms/step - loss: 0.7524 - categorical_accuracy: 0.6667
Epoch 295/540
2/2 [==============================] - 0s 39ms/step - loss: 0.7949 - categorical_accuracy: 0.5965
Epoch 296/540
2/2 [==============================] - 0s 39ms/step - loss: 0.7377 - categorical_accuracy: 0.5965
Epoch 297/540
2/2 [==============================] - 0s 38ms/step - loss: 0.7095 - categorical_accuracy: 0.7193
Epoch 298/540
2/2 [==============================] - 0s 38ms/step - loss: 0.6953 - categorical_accuracy: 0.6491
Epoch 299/540
2/2 [==============================] - 0s 38ms/step - loss: 0.6575 - categorical_accuracy: 0.5965
Epoch 300/540
2/2 [==============================] - 0s 40ms/step - loss: 0.7617 - categorical_accuracy: 0.6667
Epoch 301/540
2/2 [==============================] - 0s 39ms/step - loss: 0.6797 - categorical_accuracy: 0.6316
Epoch 

2/2 [==============================] - 0s 40ms/step - loss: 0.3757 - categorical_accuracy: 0.8596
Epoch 367/540
2/2 [==============================] - 0s 39ms/step - loss: 0.3653 - categorical_accuracy: 0.8246
Epoch 368/540
2/2 [==============================] - 0s 38ms/step - loss: 0.4864 - categorical_accuracy: 0.8070
Epoch 369/540
2/2 [==============================] - 0s 40ms/step - loss: 0.7633 - categorical_accuracy: 0.7895
Epoch 370/540
2/2 [==============================] - 0s 38ms/step - loss: 0.4769 - categorical_accuracy: 0.7544
Epoch 371/540
2/2 [==============================] - 0s 39ms/step - loss: 0.4890 - categorical_accuracy: 0.8070
Epoch 372/540
2/2 [==============================] - 0s 38ms/step - loss: 0.5969 - categorical_accuracy: 0.7368
Epoch 373/540
2/2 [==============================] - 0s 40ms/step - loss: 0.5162 - categorical_accuracy: 0.8070
Epoch 374/540
2/2 [==============================] - 0s 39ms/step - loss: 0.5428 - categorical_accuracy: 0.7544
Epoch 

2/2 [==============================] - 0s 39ms/step - loss: 0.1227 - categorical_accuracy: 0.9649
Epoch 440/540
2/2 [==============================] - 0s 38ms/step - loss: 0.1083 - categorical_accuracy: 0.9649
Epoch 441/540
2/2 [==============================] - 0s 39ms/step - loss: 0.1021 - categorical_accuracy: 0.9649
Epoch 442/540
2/2 [==============================] - 0s 42ms/step - loss: 0.0990 - categorical_accuracy: 0.9649
Epoch 443/540
2/2 [==============================] - 0s 41ms/step - loss: 0.0889 - categorical_accuracy: 0.9649
Epoch 444/540
2/2 [==============================] - 0s 39ms/step - loss: 0.0848 - categorical_accuracy: 0.9649
Epoch 445/540
2/2 [==============================] - 0s 40ms/step - loss: 0.0851 - categorical_accuracy: 0.9825
Epoch 446/540
2/2 [==============================] - 0s 40ms/step - loss: 0.0734 - categorical_accuracy: 0.9825
Epoch 447/540
2/2 [==============================] - 0s 42ms/step - loss: 0.1020 - categorical_accuracy: 0.9474
Epoch 

2/2 [==============================] - 0s 42ms/step - loss: 0.6920 - categorical_accuracy: 0.6842
Epoch 513/540
2/2 [==============================] - 0s 42ms/step - loss: 0.6662 - categorical_accuracy: 0.7193
Epoch 514/540
2/2 [==============================] - 0s 41ms/step - loss: 0.6371 - categorical_accuracy: 0.7018
Epoch 515/540
2/2 [==============================] - 0s 40ms/step - loss: 0.6067 - categorical_accuracy: 0.7193
Epoch 516/540
2/2 [==============================] - 0s 38ms/step - loss: 0.5837 - categorical_accuracy: 0.7368
Epoch 517/540
2/2 [==============================] - 0s 40ms/step - loss: 0.5336 - categorical_accuracy: 0.8421
Epoch 518/540
2/2 [==============================] - 0s 40ms/step - loss: 0.4632 - categorical_accuracy: 0.8947
Epoch 519/540
2/2 [==============================] - 0s 38ms/step - loss: 0.4241 - categorical_accuracy: 0.8947
Epoch 520/540
2/2 [==============================] - 0s 39ms/step - loss: 0.4084 - categorical_accuracy: 0.8947
Epoch 

In [56]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 237,251
Trainable params: 237,251
Non-trai

In [57]:
model.save('actions.h5')

In [58]:
model.load_weights('actions.h5')

In [59]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [60]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 357ms/step


In [61]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [62]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[1, 1],
        [0, 1]],

       [[1, 0],
        [1, 1]]], dtype=int64)

In [63]:
accuracy_score(ytrue, yhat)

0.6666666666666666

In [64]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [65]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.75

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions_temp[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions_temp[np.argmax(res)])
                else:
                    sentence.append(actions_temp[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions_temp, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 23ms/step
